<a href="https://colab.research.google.com/github/saivarun08777/saivarun08777/blob/main/nltk_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy()
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/nltk/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/nltk/test.csv')

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_data.info(), test_data.info()

In [ ]:
train_data.describe

In [ ]:
test_data.describe

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.columns

In [ ]:
test_data.columns

In [ ]:
train_data.dtypes

In [ ]:
test_data.dtypes

In [ ]:
train_data['lang_abv'].unique()

In [ ]:
train_data['language'].unique()

In [ ]:
train_data['label'].unique().sum()

In [ ]:
train_data['label'].unique()

In [ ]:
test_data['lang_abv'].unique()

In [ ]:
test_data['language'].unique()

In [ ]:
train_data=train_data.drop('lang_abv',axis=1)
test_data=test_data.drop('lang_abv',axis=1)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
sns.countplot(train_data['language'], palette='bright')
sns.set(rc={'figure.figsize':(15,15)})

In [ ]:
sns.countplot(train_data['label'], palette='deep')

In [ ]:
sns.countplot(test_data['language'], palette='bright')
sns.set(rc={'figure.figsize':(5,5)})

In [ ]:
labels, frequencies = np.unique(train_data.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels = labels, autopct = '%1.1f%%')
plt.show()

In [ ]:
labels, frequencies = np.unique(test_data.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels = labels, autopct = '%1.1f%%')
plt.show()

In [ ]:
pip install sentencepiece

In [ ]:
pip install tokenizer

In [ ]:
pip install backend tokenizer

In [ ]:
pip install slow tokenizer

In [ ]:
pip install fast tokenizer

In [ ]:
pip install serialization

In [ ]:
pip install tokenizers

In [ ]:
import tokenizer

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
len(tokenizer.vocab)

In [ ]:
def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
def bert_encode( premises,hypotheses,tokenizer):
    
    num_examples = len(hypotheses)

    sentence1 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(hypotheses)])
    sentence2 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(premises)])

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor()

    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs

In [ ]:
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
train_input = bert_encode(train_data.premise.values, train_data.hypothesis.values, tokenizer)

In [ ]:
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape = (None,),dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
with strategy.scope():
    model = build_model()
    model.summary()

In [ ]:
model.fit(train_input, train_data.label.values, epochs = 5, verbose = 1, batch_size = 16, validation_split = 0.2)

In [ ]:
test_input = bert_encode(test_data.premise.values, test_data.hypothesis.values, tokenizer)

In [ ]:
test_data.head()

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

In [ ]:
pred_data = test_data.id.copy().to_frame()
pred_data['prediction'] = predictions

In [ ]:
pred_data.head(10)

In [ ]:
pred_data.to_csv("pred_data.csv", index = False)

In [ ]:
sns.countplot(pred_data['prediction'], palette='bright')